In [1]:
from unsloth import FastLanguageModel, UnslothTrainer, UnslothTrainingArguments
from datasets import load_dataset
from transformers import LlamaTokenizer
import torch

/home/rwalling/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2024-07-09 17:41:33.386410: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-09 17:41:33.565668: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-09 17:41:33.566814: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-09 17:41:33.812379: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-09 17:41:35.268192: W tensorflow/compiler/tf

In [2]:
# Dataset
dataset = load_dataset("Eitanli/diet_type")

In [3]:
dataset["train"]['id'][100]

100

In [5]:
dataset["train"]['recipe'][100]

"Goat Cheese Cheesecake with Spiced Wafer Crust\n6 tablespoons unsalted butter, melted, plus more for the pan\n2 cups vanilla wafer crumbs, such as Nilla\n1/4 cup sugar\n1/2 teaspoon ground cinnamon\n1/2 teaspoon ground ginger\n2 grates of fresh nutmeg\nPinch kosher salt\nTwo 8-ounce packages cream cheese, at room temperature\nOne 12-ounce log goat cheese\n1 1/2 cups sour cream sour cream\n4 large eggs\n1 1/2 cups sugar\n2 teaspoons vanilla extract\nInstructions:\nFor the crust: Butter a 9-inch springform pan. In a large mixing bowl, combine the butter, wafer crumbs, sugar, cinnamon, ginger, nutmeg and salt. Stir together until well combined and homogenous. Use your fingers to press the crumb mixture onto the bottom of the pan and about halfway up the sides. Reserve.\nFor the filling: Preheat the oven to 350 degrees. In an electric mixer with a paddle attachment, beat the cream cheese and goat cheese together until light and fluffy. Add the sour cream and continue beating until combine

In [4]:
dataset["train"]['diet_type'][100]

'[(Dairy-Free,80),(Vegetarian,70),(Lactovegetarians,60),(OvoVegetarian,50),(Paleo,40)]'

In [6]:
tokenizer = LlamaTokenizer

def tokenize_function(examples):
    return tokenizer(examples["recipe", "diet_type"], padding="max_length", truncation=True)

In [7]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|               | 0/74465 [00:00<?, ? examples/s]


KeyError: ('recipe', 'diet_type')

In [8]:
max_seq_length = 2048 # Choose any
dtype = None # None for auto detection.
load_in_4bit = True # Use 4bit quantization to reduce memory usage.

# Load LoRA
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Phi-3-mini-4k-instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth: Fast Mistral patching release 2024.7
   \\   /|    GPU: NVIDIA GeForce RTX 4070 SUPER. Max memory: 11.994 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.1+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [9]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj", 
                      "lm_head"],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
)

Unsloth: Offloading output_embeddings to disk to save VRAM


Unsloth 2024.7 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Unsloth: Casting lm_head to float32
